In [1]:
%matplotlib widget
import bmcs_utils.api as bu
from bmcs_cross_section.api import ReinfLayer
from bmcs_shear.shear_crack.crack_tip_orientation import CrackStateAnimator
#from bmcs_shear.shear_crack.crack_extension import CrackStateAnimator
ce = CrackStateAnimator()
ce.sz_bd.cmm.trait_set(tau_1 = 4, s_1 = 0.2, tau_2 = 3, tau_3 = 2, s_2 = 1.4, s_3 = 5)
x_00=250
ce.sz_cp.trait_set(x_00=x_00, n_m = 15)
ce.sz_bd.Rectangle = True
ce.sz_bd.trait_set(H=300, B=200, L=500)
ce.sz_cp.add_x_tip_an([x_00-10, 50]) 
ce.sz_cp.add_x_tip_an([x_00-30, 100]) 

bl1 = ReinfLayer(name='layer1', z=44, A=ce.sz_bd.smm.A_f, matmod='steel')
bl1.matmod_.trait_set(E=ce.sz_bd.smm.E_f, f_t=ce.sz_bd.smm.sig_y)
ce.sz_bd.cross_section_layout.add_layer(bl1)
ce.sz_bd
ce.sz_cp.sz_ctr.w = 10
#ce.sz_bd.cross_section_layout.reinforcement[0].trait_set(z=20, A=1, E=1)

In [2]:
ce.interact()

<lambdifygenerated-25>:2: RuntimeWarning: overflow encountered in exp
  return (select([less(f_c, -35.7142857142857*d_a**(-0.32)*f_c**(-0.18)*f_t**2*w),less_equal(w, 0.028*d_a**0.32*f_c**0.18/f_t),True], [-f_c,35.7142857142857*d_a**(-0.32)*f_c**(-0.18)*f_t**2*w,f_t*exp(-35.7142857142857*d_a**(-0.32)*f_c**(-0.18)*f_t*(-0.028*d_a**0.32*f_c**0.18/f_t + w))], default=nan))
<lambdifygenerated-26>:2: RuntimeWarning: invalid value encountered in sqrt
  return (select([less_equal(w, 0),True], [0,0.25*f_c*s*(-sqrt(2)*sqrt(w/d_a) + 1)*((2.44 - 39.04/f_c)*abs(s/w)**3 + 9.8/f_c)/(w*(s**4*(2.44 - 39.04/f_c)/w**4 + 1))], default=nan))


In [30]:
%%capture
import numpy as np
import matplotlib.pylab as plt
from matplotlib import animation, rc

fig = plt.figure(figsize=(13,6))
fig.canvas.header_visible = False
fig_sub_plot_ = fig.add_subplot(1,1,1)

psi_range = np.linspace(0, np.pi/4, 15)
psi_range = np.hstack([psi_range, psi_range[-1:0:-1]])

ce.sz_cp.sz_ctr.x_rot_1k = 200

x_rot_1k = ce.sz_cp.sz_ctr.x_rot_1k

w_range = np.linspace(0, 40, 15)
w_range = np.hstack([w_range, w_range[-1:0:-1]])

v_n_range = np.linspace(0,1, 15)
v_n_range = np.hstack([v_n_range, v_n_range[-1:0:-1]])

load_def_range = np.linspace(0,1,15)
load_def_range = np.hstack([load_def_range, load_def_range[-1:0:-1]])


# def load_deformation_path(i):
#     ce.v_n_slider = v_n_range[i]
#     ce.load_slider = load_def_range[i]
#     fig.clf()
#     v_n = ce.

def plot_crack_path(i):
    ce.psi_slider = psi_range[i]
    #ce.x_rot_1k_slider = x_rot_1k
    ce.w_slider = w_range[i]
    fig.clf()
    axes = ce.sz_cp.subplots(fig)
    ce.sz_cp.update_plot(axes)
    axes.set_ylim(ymin=-10, ymax=320)
    axes.set_xlim(xmin = 295, xmax=296)
    

def plot_crack_tip_rotation(i):
    ce.psi_slider = psi_range[i]
    #ce.x_rot_1k_slider = x_rot_1k
    ce.w_slider = w_range[i]
    fig.clf()
    axes = ce.sz_cp.sz_ctr.subplots(fig)
    ce.sz_cp.sz_ctr.update_plot(axes)
    axes.set_xlim(xmin=50, xmax=350)
    

def plot_stress_state(i):
    ce.psi_slider = psi_range[i]
    fig.clf()
    axes = ce.sz_stress_profile.subplots(fig)
    ce.sz_stress_profile.update_plot(axes)
    ax_u_0, ax_w_0, ax_S_0, ax_F_0, ax_u_1, ax_w_1, ax_S_1, ax_F_1 = axes
    
    
def plot_deformed_state(i):
    ce.psi_slider = psi_range[i]
    ce.x_rot_1k_slider = x_rot_1k
    fig.clf()
    axes = ce.sz_stress_profile.ds.subplots(fig)
    ce.sz_stress_profile.ds.update_plot(axes)
    axes.set_ylim(ymin=-10, ymax=340)
    axes.set_xlim(xmin = 295, xmax=296)

In [31]:
def html5_crack_path_video():
    n_t = len(psi_range)
    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, plot_crack_path,
                                   frames=n_t, interval=100, blit=True)
    return anim.to_html5_video()


def html5_crack_tip_rotation_video():
    n_t = len(psi_range)
    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, plot_crack_tip_rotation,
                                   frames=n_t, interval=100, blit=True)
    return anim.to_html5_video()

def html5_stress_state_video():
    n_t = len(psi_range)
    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, plot_stress_state,
                                   frames=n_t, interval=150, blit=True)
    return anim.to_html5_video()

def html5_deformed_state_video():
    n_t = len(psi_range)
    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, plot_deformed_state,
                                   frames=n_t, interval=100, blit=True)
    return anim.to_html5_video()

In [32]:
import os.path
home_dir = os.path.expanduser('~')
anim_dir = os.path.join(home_dir, 'generated_animation')
if not os.path.exists(anim_dir):
    os.makedirs(anim_dir)

In [33]:
%%capture
crack_path_anim_file = os.path.join(anim_dir, 'crack_path.html')
with open(crack_path_anim_file,'w') as html_video_file:
    html_video_file.write(html5_crack_path_video())

In [34]:
%%capture
crack_tip_rotation_anim_file = os.path.join(anim_dir, 'crack_tip_rotation.html')
with open(crack_tip_rotation_anim_file,'w') as html_video_file:
    html_video_file.write(html5_crack_tip_rotation_video())

In [35]:
%%capture
stress_anim_file = os.path.join(anim_dir, 'stress_state_animation.html')
with open(stress_anim_file,'w') as html_video_file:
    html_video_file.write(html5_stress_state_video())

In [36]:
%%capture
deformed_anim_file = os.path.join(anim_dir, 'deformed_state_animation.html')
with open(deformed_anim_file,'w') as html_video_file:
    html_video_file.write(html5_deformed_state_video())

In [37]:
from IPython.display import HTML
html_video_file = open(deformed_anim_file,'r')
HTML(html_video_file.read())

In [38]:
from IPython.display import HTML
html_video_file = open(crack_tip_rotation_anim_file,'r')
HTML(html_video_file.read())

In [12]:
from IPython.display import HTML
html_video_file = open(crack_path_anim_file,'r')
HTML(html_video_file.read())

In [10]:
%%capture
import matplotlib.pylab as plt
import numpy as np
from matplotlib import animation, rc
from bmcs_shear.shear_crack.crack_propagation import CrackPropagation
ce = CrackPropagation()
x_00=250
ce.sz_cp.trait_set(x_00=x_00, n_m = 15)
fig, (ax, ax_F) = plt.subplots(1,2, figsize = (8, 4))
ds = ce.crack_tip_orientation.crack_tip_shear_stress.sz_stress_profile.ds
sz_cp =ce.sz_cp
x_n_a =sz_cp.x_t_Ia[0,:]
ds.plot_sz_fill(ax)
F_beam = np.array(ce.F_beam)
v_beam = np.array(ce.v_n)
ax_F.set_xlim(xmax = 1.04 * np.max(v_beam))
ax_F.set_ylim(ymax = 1.04 * np.max(F_beam))
v_n = v_beam[0]
F_n = F_beam[0]

def plot_crack_step(i):
    global x_n_a, v_n, F_n
    x_n1_a = sz_cp.x_t_Ia[i,:]
    ax.plot([x_n_a[0], x_n1_a[0]], [x_n_a[1], x_n1_a[1]], '-', color = 'black')
    x_n_a = x_n1_a
    v_n1, F_n1 = v_beam[i], F_beam[i]
    ax_F.plot([v_n, v_n1], [F_n, F_n1], '-', color = 'blue')
    v_n, F_n = v_n1, F_n1

AttributeError: 'CrackStateAnimator' object has no attribute 'crack_tip_orientation'

In [11]:
def html5_crack_step_video():
    n_t = len(sz_cp.x_t_Ia)
    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, plot_crack_step,
                                   frames=n_t, interval=100, blit=True)
    return anim.to_html5_video()

In [12]:
import os.path
home_dir = os.path.expanduser('~')
anim_dir = os.path.join(home_dir, 'generated_animation')
if not os.path.exists(anim_dir):
    os.makedirs(anim_dir)

In [13]:
%%capture
crack_propagate_file = os.path.join(anim_dir, 'crack_propagate.html')
with open(crack_propagate_file,'w') as html_video_file:
    html_video_file.write(html5_crack_step_video())

RuntimeError: The animation function must return a sequence of Artist objects.

In [14]:
from IPython.display import HTML
html_video_file = open(deformed_anim_file,'r')
HTML(html_video_file.read())

NameError: name 'deformed_anim_file' is not defined